### MATH2504 S2, 2022, BigHW Submission 
### Submission form II of IV 

---

**Note: It is expected that this notebook be runnable in the sense that the marker can select "Restart & Run All" and all code cells of solutions will execute. This requires not to have name clashes between solutions.**

---

# Solution to [Question 5](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q5)

In [ ]:
#Write a function that inputs a positive integer x, and returns True if x is a perfect number, and False otherwise
function perfect_number(n)
    sum = 1
    i = 2
    while i^2 <= n
        if n % i == 0
            sum += i + n/i
        end
        i+= 1
    end
    (sum == n && n != 1)  ? (return true) : (return false)
end


#Write a function that prints all the perfect numbers less than or equal to x
function all_perfect_numbers(x::Int, verbose::Bool = false)
    to_be_checked = vcat([i for i in 8:10:x],[i for i in 6:10:x])
    perfect_number_indexes = [perfect_number(i) for i in to_be_checked]
    return sort(to_be_checked[perfect_number_indexes])
end

all_perfect_numbers(10^6,true)

# Solution to [Question 6](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q6)

In [ ]:
using Plots
using StatsBase

function sieve_of_Eratosthenes(n)
    primebits = ones(Bool,n) #Will contain true if the index is prime (initially all assumed prime)
    primebits[1] = false #The number 1 is not prime
    p = 2 #Smallest prime
    while p ≤ n
        i = 2p
        while i ≤ n  # \le +[TAB]
            primebits[i] = false
            i += p
        end
        p += 1
        while p ≤ n && !primebits[p]
            p += 1
        end
    end
    (1:n)[primebits]
end

function check_goldbach(nmax)
    primes = sieve_of_Eratosthenes(nmax)
    num_pairs = 0 * Vector{Float64}(undef,2*nmax)
    for i in eachindex(primes)
        for j in i:length(primes)
            num_pairs[primes[i]+primes[j]] += 1
        end
    end
    return num_pairs
end

nmax = 10^6
pairs = check_goldbach(nmax)
subset = sample(4:2:nmax, 20000 ,replace= false) 
###
evens = pairs[4:2:nmax]
if 0 ∈ evens
    println("Found a counter example for Goldbach")
end
###

scatter(subset,pairs[subset],legend=false,xlabel="n",ylabel="Number of Goldbach pairs")



# Solution to [Question 7](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q7)

In [ ]:
using Random, BenchmarkTools

random_license_plate() = String(rand('0':'9',3)) * " " * String(rand('A':'Z',3))

function prepare_data(;N = 10^7)
    Random.seed!(1)
    indexes = rand(1:10^12, N)
    plates = [random_license_plate() for _ in 1:N]

    #One specific spot is set to a special value.
    special_spot = Int(floor(0.9*N)) #Arbitrary index
    indexes[special_spot], plates[special_spot] = 1234567890, "000 ZZZ"
    
    return indexes, plates
end

function prepare_sorted_data(;N = 10^7)
    indexes, plates = prepare_data(N=N)  
    perm = sortperm(indexes)
    return indexes[perm], plates[perm]
end


## 7a:

The one line function definition of a function is referred to 'assignment form' in the documentation. It is a method to define a function in Julia with a lot less clutter and noise than doing a full defintion using `function` and `end`. However, the drawback is that the function must have a single expression. 



In [ ]:
# simple assignment form function that adds two numbers
adder(x,y) = x + y

adder(1,5)

On top of multiplication, Julia uses the `*` function to concatenate strings. Concatenating strings simply takes all the elements in each string in order, and combines them into a larger string. In this case, we are concatenating the randomly generated numerical and alphabetical parts of the number plate, along with a space in the middle. 

In [ ]:
# concatenating some strings

"hello" * " "* "there"

In this code, for each use of `rand`, it is supplied with a set of items to sample from, then a number of times to sample, placing the results into a vector. For example, to generate the numeric part of the number plate, we sample 3 random digits from 0 to 9. 

In [ ]:
rand('0':'9',3)

The `[ _ for in ]` form is referred to as list comprehension. It is a one line method for defining an array based on some kind of function without resorting to a for loop. 

In [ ]:
a = [i^2 for i  in 1:10]

Returning tuples means we can return multiple outputs in the same line, separated by a comma. This is good for conciseness, but it also allows for the following syntax:

In [ ]:
x = 4
y = 5 # defining x and y
x, y = y, x # swapping their values
return x, y #returning the tuple

The `sortperm()` function is similar to the sort function. However, instead of rearranging the elements in the array to place them in the desired order, it arranges the indexes of the elements in the array in the desired order. This is useful if you want to sort elements, but also keep track of their original positions.

In [ ]:
tbs = [5,2,4,1,9,3]

index_of_positions = sortperm(tbs)

#gives the indexes of the elements to sort in ascending order

## 7b:

There is a small chance that we get a duplication of the 1234567890 index. This could be randomly generated with a $\frac{N}{10^{12}}$ chance  after N generations. There is a chance that we replace the index with itself when assignment the special index. This chance is $\frac{N - s}{N}$, where s is the number of times the special index was generated randomly. For large N, this chance is close to 1, so it is unlikely that we replace the special index with itself. 

As of the 3rd of August at 7pm, the BenchmarkTools package has 306 commits. 

On the 6th of July 2021, a user commented that the output of the `@benchmark` tool had a typo, spelling BenchmarkTools as BechmarkTools. 
This was then addressed and merged by one of the contributors. 

From the documentation, it appears that `@btime` works similarly to `@benchmark`, however it reduces the amount of output information to only the minimum time and memory allocation. `@btime` is meant to behave like the built in `@time` macro in Julia. It is also described as a convenience wrapper, meaning that if you don't need all the information provided by `@benchmark` you can just use `@btime`. 



In [6]:
function find_special_plate(indexes, plates; special_index = 1234567890)
    for (i, index) in enumerate(indexes) 
        if index == special_index 
            return plates[i]
        end
    end 
    throw(Error("Index not found"))
end

indexes, plates = prepare_sorted_data()

@benchmark find_special_plate(indexes, plates)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.012 μs …   7.024 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.137 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.140 μs ± 220.239 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▁▁ █▆▁▁▁▄▁                                                ▂
  ████▇████████▇▆▆▇▇▆▅▆█▆▆▆▆▆▇▇▇▇▇▆▆▄▆▆▆▇▆▇▄▆▅▁▅▆▅▅▆▆▅▄▅▃▆▄▃▄ █
  4.01 μs      Histogram: log(frequency) by time      5.35 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

## 7c:

In [7]:
using Random, BenchmarkTools

function my_binary_search(indexes, plates; special_index = 1234567890)
    n = length(indexes)
    if n >= 1
        m = Int((n + 1)÷2)
        if indexes[m] == special_index
            return plates[m]
        elseif indexes[m] > special_index
            return my_binary_search(indexes[1:(m-1)], plates[1:(m-1)], special_index = special_index)
        else indexes[m] < special_index
            return my_binary_search(indexes[m:n], plates[m:n],special_index = special_index )
        end
    else
        throw(("Index not found"))
    end
end

indexes, plates = prepare_sorted_data(N=5) 


@btime my_binary_search(indexes, plates; special_index = 1234567890)


  100.467 ns (2 allocations: 144 bytes)


"000 ZZZ"

## 7d:

The binary search picks the middle element of an array, then decides whether to look in the top or bottom half. It repeats this for each iteration. In the process, the length of the subset of the array it is searching decreases according to the following pattern:
$n , \frac{n}{2}, \frac{n}{2^2}, \dots \frac{n}{2^k}$. After $k$ iterations, we expect the problem to reduce to a 1 element array, which should be the solution. 
$\frac{n}{2^k} = 1 \to n = 2^k \to k = \log_2 n$, which shows that at worst the time complexity is $O( \log n)$.

# Solution to [Question 8](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q8)

In [ ]:
function array_table(array, heading)
    println(heading,":")
    for (i,a) in enumerate(array)
        println(i,"\t",a)
    end
end

## 8a:

In [ ]:
name_list = [   "Amy Chan",
            "Maithili Mehta",
            "Anna Foeglein",
            "Andy Ferris",
            "Thomas Graham",
            "Elaine Schenk",
            "Jesse Woods",
            "Tina Moghaddam",
            "Paul Bellette",
            "Paul Vrbik",
            "Tom Cranitch",
            "Yoni Nazarathy",
            "Sam Hambleton",
            "Alistair Falconer",
            "Emma Comino",
            "Ivana Carrizo-Molina"]

array_table(sort(name_list, by = (x) -> length(split(x," ")[2])),"Sorted by length of last name (Base Sort)")

## 8b:

In [ ]:

function my_sort_bubble!(A; by = (x) -> x)
    n = length(A)
    for i in 1:n
        for j in 1:(n - i)
            if by(A[j]) > by(A[j+1])
                A[j] , A[j+1] = A[j+1] , A[j]
            end
        end
    end
end
            
name_list = [   "Amy Chan",
            "Maithili Mehta",
            "Anna Foeglein",
            "Andy Ferris",
            "Thomas Graham",
            "Elaine Schenk",
            "Jesse Woods",
            "Tina Moghaddam",
            "Paul Bellette",
            "Paul Vrbik",
            "Tom Cranitch",
            "Yoni Nazarathy",
            "Sam Hambleton",
            "Alistair Falconer",
            "Emma Comino",
            "Ivana Carrizo-Molina"]

my_sort_bubble!(name_list, by = (x) -> length(split(x," ")[2]))

array_table(name_list,"Sorted by length of last name (Bubble Sort)")



## 8c:

In [ ]:

function quick_sort!(A; by = (x) -> x)
    qsort(A, 1, length(A), by = by)
end

function qsort(A, low, high; by = (x) -> x)
    if low < high
        part = parter(A, low, high, by = by)
        qsort(A, low, part - 1, by = by)
        qsort(A, part + 1, high, by = by)
    end
end

function parter(A, low, high; by = (x) -> x)
    m = (low + high) ÷ 2
    A[m] , A[high] = A[high], A[m]
    pivot = by(A[high])
    #A[m], A[high] = A[high], A[m]
    store = low
    for i = low:(high-1)
        if by(A[i]) <= pivot
            A[i], A[store] = A[store], A[i]
            store += 1
        end
    end
    A[high], A[store] = A[store], A[high]
    return store
end

            
name_list = [   "Amy Chan",
            "Maithili Mehta",
            "Anna Foeglein",
            "Andy Ferris",
            "Thomas Graham",
            "Elaine Schenk",
            "Jesse Woods",
            "Tina Moghaddam",
            "Paul Bellette",
            "Paul Vrbik",
            "Tom Cranitch",
            "Yoni Nazarathy",
            "Sam Hambleton",
            "Alistair Falconer",
            "Emma Comino",
            "Ivana Carrizo-Molina"]

quick_sort!(name_list, by = (x) -> length(split(x," ")[2]))


array_table(name_list,"Sorted by length of last name (Quick Sort)")


## 8d:

In [ ]:
using BenchmarkTools
println("Bubble Sort with 10^4 random integers")
@btime my_sort_bubble(rand(Int,10^4)) ;
println("Bubble Sort with 10^5 random integers")
@btime my_sort_bubble(rand(Int,10^5)) ;
println("Quick Sort with 10^6 random integers")
@btime quick_sort!(rand(Int, 10^6)) ;
println("Quick Sort with 10^7 random integers")
@btime quick_sort!(rand(Int, 10^7)) ;
println("Quick Sort with 10^8 random integers")
@btime quick_sort!(rand(Int, 10^8)) ;
println("Base sort with 10^6 random integers")
@btime sort(rand(Int,10^6)) ;
println("Base sort with 10^7 random integers")
@btime sort(rand(Int,10^7)) ;
println("Base sort with 10^8 random integers")
@btime sort(rand(Int,10^8)) ;

# Solution to [Question 9](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q9)

## 9a:

In [ ]:
using LinearAlgebra
using Random
Random.seed!(0)
A = round.(10*rand(2,3))
B = round.(10*rand(3,4))

function my_mult_by_cols(A,B)
    rowsA, columnsA = size(A)
    rowsB,columnsB = size(B)
    @assert columnsA == rowsB
    C = Array{Float64}(undef, rowsA, 0)
    new_column = Array{Float64}(undef, rowsA, 1)
    A_columns = [A[:,x] for x in 1:columnsA] #Split A into its column vectors
    
    for i in 1:columnsB
        new_entry = [A_columns[j]*B[j,i] for j in 1:rowsB]
        new_column = sum(new_entry)
        C = hcat(C, new_column) 
    end
    return C
end

my_mult_by_cols(A,B)

## 9b:

In [ ]:
function my_mult_by_rows(A,B)
    rowsA, columnsA = size(A)
    rowsB,columnsB = size(B)
    @assert columnsA == rowsB
    C = Array{Float64}(undef, 0, columnsB)
    new_row = Array{Float64}(undef, 1, columnsB)
    B_rows = [B[x,:] for x in 1:rowsB] #Split B into its row vectors
    
    for i in 1:rowsA
        new_entry = [B_rows[j]*A[i,j] for j in 1:columnsA]
        new_row = transpose(sum(new_entry)) #transpose() prevents new_row being a column vector
        C = vcat(C, new_row) 
    end
    return C
end

my_mult_by_rows(A,B)

## 9c:

In [ ]:
function my_mult_sum_outer_products(A,B)
    rowsA, columnsA = size(A)
    rowsB,columnsB = size(B)
    @assert columnsA == rowsB

    AB = [A[:,j]*transpose(B[j,:]) for j in 1:columnsA]
    C = sum(AB)
end

my_mult_sum_outer_products(A,B)

## 9d:


In my_mult_inner_products, we have a nested loop that involves $n\times m$ calculations where $n$ is the number of rows in $A$ and $m$ is the number of columns in $B$ (and where $n\times m$ are the dimensions of $C$). In this case, each individual element of C is being calculated. For 9a and b, $C$ is being calculated per column/row, so only $m$ calculations are needed for 9a and $n$ calculations are needed for 9b. Evidently, these two methods involve fewer calculations than the inner product. Finally, 9c finds the matrix $C$ using the outer product. My code shows that only $m$ iterations are needed to calculate an array of vectors, and then sum those vectors to get $C$, so overall there are $m+1$ calculations.

Therefore, in terms of the number of operations, my_mult_inner_product has more operations involved than the functions in 9a,b and c. 